Task A: Binary Classification Using Handcrafted Features and ML Classifiers

In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.feature import hog, local_binary_pattern
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV

def load_images(folder):
    images, labels = [], []
    for category in ["with_mask", "without_mask"]:
        path = os.path.join(folder, category)
        label = 1 if category == "with_mask" else 0
        for file in os.listdir(path):
            img_path = os.path.join(path, file)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img = cv2.resize(img, (64, 64))
                img = cv2.equalizeHist(img)  # Apply Histogram Equalization
                images.append(img)
                labels.append(label)
    return np.array(images), np.array(labels)

def extract_features(images):
    features = []
    for img in images:
        hog_feature = hog(img, pixels_per_cell=(8, 8), cells_per_block=(2, 2), feature_vector=True)
        lbp_feature = local_binary_pattern(img, P=8, R=1, method='uniform').flatten()
        canny_edges = cv2.Canny(img, 100, 200).flatten()
        
        # Sobel Edge Detection
        sobel_x = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3).flatten()
        sobel_y = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3).flatten()
        
        # Combine all features
        feature_vector = np.hstack([hog_feature, lbp_feature, canny_edges, sobel_x, sobel_y])
        features.append(feature_vector)
    return np.array(features)

dataset_path = "Face-Mask-Detection/dataset"
X, y = load_images(dataset_path)
X_features = extract_features(X)

scaler = StandardScaler()
X_features = scaler.fit_transform(X_features)

pca = PCA(n_components=100)
X_pca = pca.fit_transform(X_features)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for train_index, test_index in skf.split(X_pca, y):
    X_train, X_test = X_pca[train_index], X_pca[test_index]
    y_train, y_test = y[train_index], y[test_index]

param_grid_svm = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
svm = GridSearchCV(SVC(), param_grid_svm, cv=3)
svm.fit(X_train, y_train)
y_pred_svm = svm.best_estimator_.predict(X_test)

param_grid_mlp = {'hidden_layer_sizes': [(50,), (100,), (50, 50)], 'activation': ['relu', 'tanh'], 'max_iter': [300, 500]}
mlp = GridSearchCV(MLPClassifier(), param_grid_mlp, cv=3)
mlp.fit(X_train, y_train)
y_pred_mlp = mlp.best_estimator_.predict(X_test)

print("Best SVM Accuracy:", accuracy_score(y_test, y_pred_svm))
print("Best MLP Accuracy:", accuracy_score(y_test, y_pred_mlp))
print("\nSVM Report:\n", classification_report(y_test, y_pred_svm))
print("\nMLP Report:\n", classification_report(y_test, y_pred_mlp))


Best SVM Accuracy: 0.9201474201474201
Best MLP Accuracy: 0.9004914004914005

SVM Report:
               precision    recall  f1-score   support

           0       0.95      0.88      0.91       386
           1       0.90      0.96      0.93       428

    accuracy                           0.92       814
   macro avg       0.92      0.92      0.92       814
weighted avg       0.92      0.92      0.92       814


MLP Report:
               precision    recall  f1-score   support

           0       0.90      0.89      0.89       386
           1       0.90      0.91      0.91       428

    accuracy                           0.90       814
   macro avg       0.90      0.90      0.90       814
weighted avg       0.90      0.90      0.90       814



Task B : Face Mask Detection Using Convolutional Neural Networks (CNN)

In [13]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split

def load_images(folder):
    images, labels = [], []
    for category in ["with_mask", "without_mask"]:
        path = os.path.join(folder, category)
        label = 1 if category == "with_mask" else 0
        for file in os.listdir(path):
            img_path = os.path.join(path, file)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img = cv2.resize(img, (64, 64))
                img = cv2.equalizeHist(img)  # Apply Histogram Equalization
                images.append(img)
                labels.append(label)
    return np.array(images), np.array(labels)

def build_cnn(optimizer='adam', dropout_rate=0.5, learning_rate=1e-3):
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=(64, 64, 1)),
        MaxPooling2D(2,2),
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Conv2D(128, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(dropout_rate),
        Dense(1, activation='sigmoid')
    ])
    
    if optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        opt = SGD(learning_rate=learning_rate)
    else:
        raise ValueError("Unsupported optimizer! Choose 'adam' or 'sgd'.")
    
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Load dataset
dataset_path = "Face-Mask-Detection/dataset"
X, y = load_images(dataset_path)
X = X.reshape(-1, 64, 64, 1) / 255.0  # Normalize and reshape
xTrainCNN, xTestCNN, yTrainCNN, yTestCNN = train_test_split(X, y, test_size=0.2, random_state=42)

# Hyperparameter variations
optimizers = ['adam', 'sgd']
dropoutValues = [0.3, 0.5]
learningRates = [1e-3, 1e-4]

for opt in optimizers:
    for dropout in dropoutValues:
        for learning in learningRates:
            print(f"Training CNN with optimizer={opt}, dropout={dropout}, learning_rate={learning}")
            model = build_cnn(optimizer=opt, dropout_rate=dropout, learning_rate=learning)
            
            callbacks = [
                EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
                ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-5)
            ]
            
            model.fit(xTrainCNN, yTrainCNN, validation_data=(xTestCNN, yTestCNN), epochs=20, callbacks=callbacks, verbose=1)
            model.save(f'face_mask_classifier_{opt}_{dropout}_{learning}.keras')

# Evaluate Models
for opt in optimizers:
    for dropout in dropoutValues:
        for learning in learningRates:
            model = tf.keras.models.load_model(f'face_mask_classifier_{opt}_{dropout}_{learning}.keras')
            testLoss, testAccuracy = model.evaluate(xTestCNN, yTestCNN)
            print(f"CNN (Optimizer={opt}, Dropout={dropout}, Learning Rate={learning}) Accuracy: {testAccuracy:.4f}")


Training CNN with optimizer=adam, dropout=0.3, learning_rate=0.001
Epoch 1/20


C:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


102/102 ━━━━━━━━━━━━━━━━━━━━ 6s 42ms/step - accuracy: 0.5612 - loss: 0.6744 - val_accuracy: 0.7730 - val_loss: 0.4775 - learning_rate: 0.0010
Epoch 2/20
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 38ms/step - accuracy: 0.7973 - loss: 0.4245 - val_accuracy: 0.8736 - val_loss: 0.3359 - learning_rate: 0.0010
Epoch 3/20
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 39ms/step - accuracy: 0.8674 - loss: 0.3106 - val_accuracy: 0.9141 - val_loss: 0.2240 - learning_rate: 0.0010
Epoch 4/20
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 38ms/step - accuracy: 0.9077 - loss: 0.2166 - val_accuracy: 0.8994 - val_loss: 0.2375 - learning_rate: 0.0010
Epoch 5/20
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 38ms/step - accuracy: 0.9236 - loss: 0.1804 - val_accuracy: 0.9252 - val_loss: 0.1889 - learning_rate: 0.0010
Epoch 6/20
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 38ms/step - accuracy: 0.9398 - loss: 0.1447 - val_accuracy: 0.9141 - val_loss: 0.2101 - learning_rate: 0.0010
Epoch 7/20
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 38ms/step - accuracy: 0.9512 - loss: 0.1263 - val_